In [297]:
import pandas as pd
import requests
import json

In [298]:
df = pd.read_csv('../data/allocine_gold.csv') 

In [299]:
df = df.head()

In [300]:
import numpy as np

# Appliquer le logarithme naturel à la colonne 'actors_popularity'
df['actor_1_popularity'] = round(np.log1p(df['actor_1_popularity']), 6)
df['actor_2_popularity'] = round(np.log1p(df['actor_2_popularity']), 6)
df['actor_3_popularity'] = round(np.log1p(df['actor_3_popularity']), 6)
df['actors_popularity'] = round(np.log1p(df['actors_popularity']), 6)
df['director_popularity'] = round(np.log1p(df['director_popularity']), 6)
df['duree'] = round(np.log1p(df['duree']), 6)
# df['reputation_distributeur_1'] = round(np.log1p(df['reputation_distributeur_1']), 6)
# df['nombre_films_distributeur'] = round(np.log1p(df['nombre_films_distributeur']), 6)
df['moyenne_entrees_par_distributeur'] = round(np.log1p(df['moyenne_entrees_par_distributeur']), 6)

#df['budget'] = round(np.log1p(df['budget']), 6)
#df['budget'] = df['budget'] / df['budget'].max()

In [301]:
# # Liste des noms de colonnes à modifier
# columns_to_rename = ['vacances','saison','pays','annee_production','type','genre','genre1','langue',
#                      'actor_1_popularity', 'actor_2_popularity', 'actor_3_popularity', 'duree',
#                      'director_popularity',  'nombre_films_distributeur', 'budget','reputation_distributeur']

# # Renommer les colonnes en ajoutant des guillemets doubles
# for column in columns_to_rename:
#     new_column_name = f'"{column}"'
#     df.rename(columns={column: new_column_name}, inplace=True)


In [302]:
df

,titre,acteur_1,acteur_2,acteur_3,realisateur,distributeur,note_presse,duree,genre,pays,...,vacances,saison,actors_popularity,langue,budget,genre1,genre2,moyenne_entrees_par_distributeur,reputation_distributeur,nombre_films_distributeur
0,L'Amour et les Forêts,Virginie Efira,Melvil Poupaud,Dominique Reymond,Valérie Donzelli,Diaphana Distribution,3.800000,4.663439,Thriller,France,...,False,Printemps,3.687178,fr,0.0,Drama,Romance,11.268638,362.0,125
1,Monsieur Constant,Jean-Claude Drouot,Cali,Danièle Evenou,Alan Simon,Babaika,2.300000,4.691348,Comédie dramatique,France,...,False,Printemps,1.911762,fr,0.0,Comedy,Drama,3.713572,18.0,1
2,Les Herbes sèches,Deniz Celiloğlu,Merve Dizdar,Musab Ekici,Nuri Bilge Ceylan,Memento Distribution,4.000000,5.288267,Drame,Turquie,...,True,Été,3.158319,tr,3720000.0,Drama,Drama,11.160235,355.0,82
3,Carmen,Paul Mescal,Melissa Barrera,Rossy de Palma,Benjamin Millepied,Pathé,2.500000,4.762174,Drame,Australie,...,False,Été,4.079434,en,0.0,Music,Drama,12.753120,424.0,241
4,Carry On Jatta 3,Gippy Grewal,Sonam Bajwa,Binnu Dhillon,Smeep Kang,Friday Entertainment,3.082899,4.948760,Comédie,Inde,...,False,Été,2.890316,pa,0.0,Drama,Comedy,9.443292,272.0,10


In [303]:
def prediction_page(data):
    categorical_features = ["vacances", "saison", "pays", "annee_production", "type", "genre", "genre1", "langue"]

    numeric_features = ["actor_1_popularity", "actor_2_popularity",
    "actor_3_popularity", "duree", "director_popularity", "nombre_films_distributeur", "budget", "reputation_distributeur"]

    X = data[categorical_features + numeric_features]
    
    is_data_empty = True
    response_data = None  # Initial response data

    if not X.empty:
        is_data_empty = False

        input_data = []
        for index, row in X.iterrows():
            input_data.append({
                "duree": row['duree'],
                "genre": row['genre'],
                "pays": row['pays'],
                "type": row['type'],
                "annee_production": row['annee_production'],
                "actor_1_popularity": row['actor_1_popularity'],
                "actor_2_popularity": row['actor_2_popularity'],
                "actor_3_popularity": row['actor_3_popularity'],
                "director_popularity": row['director_popularity'],
                "vacances": row['vacances'],
                "saison": row['saison'],
                "langue": row['langue'],
                "budget": row['budget'],
                "genre1": row['genre1'],
                "reputation_distributeur": row['reputation_distributeur'],
                "nombre_films_distributeur": row['nombre_films_distributeur']
            })
        
        #print(input_data_json)
        
        api_url = "http://20.40.149.71/predict/"
        response_data = []
        for elt in input_data:
            response = requests.post(api_url, json = elt)
            
            
            #print(response.status_code)
            
            
            #if response.status_code == 200:
            response_data.append( response.json() ) # Store the response data

    return response_data  # Return the response data or None

In [304]:
print(prediction_page(df.head(3)))

[{'prediction': 109226}, {'prediction': 15598}, {'prediction': 52045}]


In [305]:
predictions_df = pd.DataFrame(prediction_page(df.head(10)))

In [306]:


# Ajouter la colonne "prediction" au DataFrame existant
df['prediction'] = predictions_df['prediction']

# Afficher le DataFrame mis à jour
print(df)

                   titre            acteur_1         acteur_2  \
0  L'Amour et les Forêts      Virginie Efira   Melvil Poupaud   
1      Monsieur Constant  Jean-Claude Drouot             Cali   
2      Les Herbes sèches     Deniz Celiloğlu     Merve Dizdar   
3                 Carmen         Paul Mescal  Melissa Barrera   
4       Carry On Jatta 3        Gippy Grewal      Sonam Bajwa   

            acteur_3         realisateur             distributeur  \
0  Dominique Reymond    Valérie Donzelli   Diaphana Distribution    
1     Danièle Evenou          Alan Simon                 Babaika    
2        Musab Ekici   Nuri Bilge Ceylan    Memento Distribution    
3     Rossy de Palma  Benjamin Millepied                   Pathé    
4      Binnu Dhillon          Smeep Kang    Friday Entertainment    

   note_presse     duree               genre       pays  ...     saison  \
0     3.800000  4.663439            Thriller     France  ...  Printemps   
1     2.300000  4.691348  Comédie dramatique